In [2]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install finta


Note: you may need to restart the kernel to use updated packages.


In [39]:
from pathlib import Path
import pandas as pd
import os
from finta import TA
import yfinance as yf
import datetime as dt
import numpy as np
import pytz
import hvplot.pandas
import seaborn as sns
%matplotlib inline

In [ ]:
## 5 records x X of variables

In [ ]:
## Sharpe ratio, Sortino ratio, Alpha, Standard deviation, variance, covariance

In [17]:
tickr='NIO'
tickrs = ["META","AMZN","GOOG","AAPL","NFLX"]
mkttickr = "^GSPC"


In [18]:
stock = yf.Ticker(tickr)
portfolio = yf.Tickers(tickrs)
mkt = yf.Ticker(mkttickr)


In [19]:
#ticker names to rename columns in dfs
ticker_names = [tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],"SP500"]


In [20]:
#historical data dataframes

stock_history_df = stock.history(period = "5y")
portfolio_history_df = portfolio.history(period = "5y")
mkt_history_df = mkt.history(period = "5y")



[*********************100%***********************]  5 of 5 completed


In [21]:
#prices df
prices_df = pd.concat([
     stock_history_df["Close"],portfolio_history_df["Close"],mkt_history_df["Close"]],
     axis = 'columns',
     join = 'inner'
    )

#renaming columns, plotting prices
prices_df.columns = [ticker_names[0],ticker_names[1],ticker_names[2],ticker_names[3],ticker_names[4],ticker_names[5],ticker_names[6]]

prices_df

,NIO,META,AMZN,GOOG,AAPL,NFLX,SP500
Date,,,,,,,
2018-09-12 00:00:00-04:00,6.60,53.245560,99.500000,58.140999,162.000000,369.950012,2888.919922
2018-09-13 00:00:00-04:00,11.60,54.531712,99.493500,58.766499,161.360001,368.149994,2904.179932
2018-09-14 00:00:00-04:00,9.90,53.912727,98.509499,58.626499,162.320007,364.559998,2904.979980
2018-09-17 00:00:00-04:00,8.50,52.477230,95.401497,57.802502,160.580002,350.350006,2888.800049
2018-09-18 00:00:00-04:00,7.68,52.563942,97.052498,58.061001,160.300003,367.649994,2904.310059
...,...,...,...,...,...,...,...
2022-10-10 00:00:00-04:00,13.28,140.419998,113.669998,98.709999,133.789993,229.979996,3612.389893
2022-10-11 00:00:00-04:00,12.83,138.979996,112.209999,98.050003,128.539993,214.289993,3588.840088
2022-10-12 00:00:00-04:00,12.88,138.339996,112.900002,98.300003,127.500000,220.869995,3577.030029


In [22]:
# combined returns dataframe (close price, daily, 5 years)

#combine data frames using only Close column
returns_df = pd.concat([
     stock_history_df["Close"],portfolio_history_df["Close"],mkt_history_df["Close"]],
     axis = 'columns',
     join = 'inner'
)
#renaming columns
returns_df.columns = [ticker_names[0],ticker_names[1],ticker_names[2],ticker_names[3],ticker_names[4],ticker_names[5],ticker_names[6]]

#calculate returns
returns_df = returns_df.pct_change().dropna() 

returns_df

,NIO,META,AMZN,GOOG,AAPL,NFLX,SP500
Date,,,,,,,
2018-09-13 00:00:00-04:00,0.757576,0.024155,-0.000065,0.010758,-0.003951,-0.004866,0.005282
2018-09-14 00:00:00-04:00,-0.146552,-0.011351,-0.009890,-0.002382,0.005949,-0.009751,0.000275
2018-09-17 00:00:00-04:00,-0.141414,-0.026626,-0.031550,-0.014055,-0.010720,-0.038978,-0.005570
2018-09-18 00:00:00-04:00,-0.096471,0.001652,0.017306,0.004472,-0.001744,0.049379,0.005369
2018-09-19 00:00:00-04:00,0.106771,0.000596,-0.007537,0.008500,0.017218,-0.001877,0.001253
...,...,...,...,...,...,...,...
2022-10-10 00:00:00-04:00,-0.034884,0.002356,-0.007769,-0.008637,0.002548,0.023270,-0.007492
2022-10-11 00:00:00-04:00,-0.033886,-0.010255,-0.012844,-0.006686,-0.039241,-0.068223,-0.006519
2022-10-12 00:00:00-04:00,0.003897,-0.004605,0.006149,0.002550,-0.008091,0.030706,-0.003291


In [23]:
std_devs = returns_df.std()
std_devs.sort_values()

SP500    0.014371
GOOG     0.019835
META     0.021753
AMZN     0.022847
AAPL     0.026313
NFLX     0.030253
NIO      0.062924
dtype: float64

In [24]:
number_of_trading_days = 252
annualized_std_devs = std_devs*np.sqrt(number_of_trading_days)
annualized_std_devs.sort_values()

SP500    0.228136
GOOG     0.314878
META     0.345314
AMZN     0.362682
AAPL     0.417713
NFLX     0.480255
NIO      0.998889
dtype: float64

In [29]:
rolling_std21ds = returns_df.rolling(window=21).std().dropna()
print(rolling_std21ds)

                                NIO      META      AMZN      GOOG      AAPL  \
Date                                                                          
2018-10-11 00:00:00-04:00  0.189825  0.016693  0.018822  0.015128  0.016692   
2018-10-12 00:00:00-04:00  0.085997  0.017734  0.021419  0.016402  0.016724   
2018-10-15 00:00:00-04:00  0.081768  0.018163  0.021519  0.016663  0.016622   
2018-10-16 00:00:00-04:00  0.076120  0.017864  0.022195  0.017673  0.018323   
2018-10-17 00:00:00-04:00  0.073321  0.017902  0.021846  0.017639  0.018345   
...                             ...       ...       ...       ...       ...   
2022-10-10 00:00:00-04:00  0.056228  0.025527  0.027405  0.020929  0.028381   
2022-10-11 00:00:00-04:00  0.044385  0.023499  0.026481  0.020877  0.028950   
2022-10-12 00:00:00-04:00  0.044167  0.020354  0.022552  0.017098  0.022187   
2022-10-13 00:00:00-04:00  0.044024  0.021828  0.022141  0.017431  0.023139   
2022-10-14 00:00:00-04:00  0.045668  0.022476  0.024

In [31]:
# Stock Covariance Calculation into a df
covariance_df = returns_df.cov()
covariance_df = covariance_df[["SP500"]]
covariance_df.columns = ["Covariance"]
# display(covariance_df)
print(covariance_df)

       Covariance
NIO      0.000270
META     0.000252
AMZN     0.000220
GOOG     0.000227
AAPL     0.000253
NFLX     0.000220
SP500    0.000207


In [33]:
#variance calculation
variance = returns_df['SP500'].var()
# print(variance)
print(variance)


0.0002065315486602782


In [35]:
# Beta calculation into a df
beta_df = returns_df.cov()/variance
beta_df = beta_df[["SP500"]]
beta_df.columns = ["Beta"]
# beta_df
print(beta_df)

           Beta
NIO    1.308343
META   1.221694
AMZN   1.065428
GOOG   1.098816
AAPL   1.224917
NFLX   1.066770
SP500  1.000000


In [36]:
# Calculate the annual average return data for the for the portfolio and the S&P 500
# Review the annual average returns sorted from lowest to highest
annualized_average_returns = returns_df.mean()*number_of_trading_days
annualized_average_returns.sort_values()

NFLX     0.004382
AAPL     0.028898
SP500    0.078855
AMZN     0.083360
GOOG     0.175235
META     0.293492
NIO      0.599924
dtype: float64

In [37]:
# Calculate the annualized Sharpe Ratios for the portfolio and the S&P 500.
# Review the Sharpe ratios sorted lowest to highest
sharpe_ratios = annualized_average_returns/annualized_std_devs
sharpe_ratios.sort_values()

NFLX     0.009124
AAPL     0.069180
AMZN     0.229845
SP500    0.345647
GOOG     0.556516
NIO      0.600591
META     0.849927
dtype: float64

In [38]:
#Create df starting by correlation
ratios_df = returns_df.corr()
ratios_df.drop(ratios_df.columns[0:-1],axis=1,inplace = True) #drop columns for correlation against each other, leave only correlation against SP500
ratios_df.rename(columns = {"SP500":"Correlation"},inplace = True) #rename column from SP500 to Correlation

#calculate additional ratios 
ratios_df["Volatility"] = returns_df.std() * np.sqrt(252)
ratios_df["Sharpe"] =  (returns_df.mean() * 252) / (returns_df.std() * np.sqrt(252))
ratios_df["Sortino"] = (returns_df.mean() * 252) / (returns_df[returns_df<0].std() * np.sqrt(252))

#add covariance and beta from calculation in cell above
ratios_df["Covariance"] = covariance_df["Covariance"]
ratios_df["Beta"] = beta_df["Beta"]
ratios_df

,Correlation,Volatility,Sharpe,Sortino,Covariance,Beta
NIO,0.298812,0.998889,0.600591,1.195495,0.000270,1.308343
META,0.807126,0.345314,0.849927,1.159902,0.000252,1.221694
AMZN,0.670181,0.362682,0.229845,0.321600,0.000220,1.065428
GOOG,0.796115,0.314878,0.556516,0.763351,0.000227,1.098816
AAPL,0.668994,0.417713,0.069180,0.088456,0.000253,1.224917
NFLX,0.506748,0.480255,0.009124,0.011179,0.000220,1.066770
SP500,1.000000,0.228136,0.345647,0.412678,0.000207,1.000000


In [82]:
## combine dfs into current_portfolio_df matching Esteban's data requirements
current_portfolio_df = pd.concat([
    prices_df["Close"], returns_df["Close"], ratios_df["Correlation", "Volatility", "Sharpe", "Sortino", "Covariance", "Beta"]],
    axis='columns',
    join='inner'
)

KeyError: 'Close'

In [83]:
## MC simulation attempt1
def __init__(self, portfolio_history_df, weights="", num_simulation=1000, num_trading_days=252):
    if not isinstance(portfolio_history_df, pd.DataFrame):
            raise TypeError("portfolio_data must be a Pandas DataFrame")
    if weights == "":
            num_stocks = len(portfolio_history_df.columns.get_level_values(0).unique())
            weights = [1.0/num_stocks for s in range(0,num_stocks)]
    else:
            if round(sum(weights),2) < .99:
                raise AttributeError("Sum of portfolio weights must equal one.")
    if not "daily_return" in portfolio_data.columns.get_level_values(1).unique():
            close_df = portfolio_data.xs('close',level=1,axis=1).pct_change()
            tickers = portfolio_data.columns.get_level_values(0).unique()
            column_names = [(x,"daily_return") for x in tickers]
            close_df.columns = pd.MultiIndex.from_tuples(column_names)
            portfolio_data = portfolio_data.merge(close_df,left_index=True,right_index=True).reindex(columns=tickers,level=0) 
    self.portfolio_data = portfolio_data
    self.weights = weights
    self.nSim = num_simulation
    self.nTrading = num_trading_days
    self.simulated_return = ""

In [78]:
 def calc_cumulative_return(self):
        last_prices = self.portfolio_history_df.xs('close',level=1,axis=1)[-1:].values.tolist()[0]
        daily_returns = self.portfolio_history_df.xs('daily_return',level=1,axis=1)
        mean_returns = returns_df.mean().tolist()
        std_returns = returns_df.std().tolist()
        portfolio_cumulative_returns = pd.DataFrame()
        for n in range(self.nSim):
        
            if n % 10 == 0:
                print(f"Running Monte Carlo simulation number {n}.")
            simvals = [[p] for p in last_prices]
            for s in range(len(last_prices)):
                for i in range(self.nTrading):
                    simvals[s].append(simvals[s][-1] * (1 + np.random.normal(mean_returns[s], std_returns[s])))
            sim_df = pd.DataFrame(simvals).T.pct_change()
            sim_df = sim_df.dot(self.weights)
            portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()
        self.simulated_return = portfolio_cumulative_returns
        self.confidence_interval = portfolio_cumulative_returns.iloc[-1, :].quantile(q=[0.025, 0.975])
        return portfolio_cumulative_returns

In [81]:
def plot_simulation(self):
    if not isinstance(self.simulated_return,pd.DataFrame):
            self.calc_cumulative_return()
    plot_title = f"{self.nSim} Simulations of Cumulative Portfolio Return Trajectories Over the Next {self.nTrading} Trading Days."
    return self.simulated_return.plot(legend=None,title=plot_title)

In [ ]:
def plot_distribution(self):
    if not isinstance(self.simulated_return,pd.DataFrame):
            self.calc_cumulative_return()
    plot_title = f"Distribution of Final Cumuluative Returns Across All {self.nSim} Simulations"
    plt = self.simulated_return.iloc[-1, :].plot(kind='hist', bins=10,density=True,title=plot_title)
    plt.axvline(self.confidence_interval.iloc[0], color='r')
    plt.axvline(self.confidence_interval.iloc[1], color='r')
    return plt

In [ ]:
def summarize_cumulative_return(self):
    if not isinstance(self.simulated_return,pd.DataFrame):
            self.calc_cumulative_return()    
    metrics = self.simulated_return.iloc[-1].describe()
    ci_series = self.confidence_interval
    ci_series.index = ["95% CI Lower","95% CI Upper"]
    return metrics.append(ci_series)

In [14]:
def perform_portfolio_analysis(user_df, portfolio_df):
    
    # get the list of all stock names in the user portfolio and also add SPY in it
     tickrs = portfolio_df['ticker'].tolist()
    tickrs.append('SPY')
    
     # use yahoo finance to retrieve 5 yrs of price data
            
    # use pct_change to convert into returns
    
    # once u have returns - u can calculate beta (risk free return / std dev), std, cumulative returns (cumprod), variance/covariance(.var() and .cov(),
    # sharpe ratio, sortino ratio
    # monte carlo output from liset's function
    
    
    
    # put all these results in dataframe(s) in a format agreed with Esteban
    # call Report function from Esteban's file
    
    return True
    
    
    
    
    
    
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)